load data

In [64]:
import pandas as pd

dataset = pd.read_csv("dataset - Merge.csv")
dataset

,Question,Unnamed: 1,Answer,Catagory
0,Katunyou AI คืออะไร,NaN,Katunyou AI คือ AI เพื่อนคู่ใจสำหรับผู้สูงอายุ...,ทั่วไป
1,Katunyou AI ช่วยเหลือผู้สูงอายุ ได้อย่างไรบ้าง,NaN,Katunyou AI ช่วยตอบคำถาม ให้คำปรึกษา เตือนความ...,ทั่วไป
2,Katunyou AI ทำอะไรได้บ้าง,NaN,Katunyou AI สามารถช่วยตอบคำถาม ให้คำปรึกษา เตื...,ทั่วไป
3,Katunyou ใช้ทำอะไร,NaN,Katunyou AI ระบบวิเคราะห์สุขภาพ และเฝ้าระวังเห...,ทั่วไป
4,Katunyou ใช้ทำอะไร,NaN,Katunyou ใช้เป็นเพื่อนคู่ใจสำหรับผู้สูงอายุ ช่...,ทั่วไป
...,...,...,...,...
242,อาหารที่ผู้สูงอายุทานได้,NaN,ผู้สูงอายุควรทานอาหารที่ย่อยง่าย เช่น ข้าวต้ม ...,อาหาร น้ำ
243,อาหารที่มีน้ำตาลสูงควรถูกจำกัดอย่างไรในผู้สูงอ...,NaN,ควรจำกัดการบริโภคอาหารที่มีน้ำตาลสูง เพราะอาจท...,อาหาร น้ำ
244,อาหารที่มีไฟเบอร์สูงดีต่อผู้สูงอายุอย่างไร?,NaN,อาหารที่มีไฟเบอร์สูงช่วยในเรื่องของการขับถ่าย ...,อาหาร น้ำ
245,อาหารเสริมที่เหมาะสมสำหรับผู้สูงอายุคืออะไร?,NaN,อาหารเสริมที่เหมาะสมสำหรับผู้สูงอายุอาจประกอบด...,อาหาร น้ำ


prepare data

In [65]:
dataset = dataset.dropna(axis=1)

In [66]:
my_cats = ["สุขภาพ", "อาหาร น้ำ"]
health_set = dataset[dataset["Catagory"]==my_cats[0]]
food_set = dataset[dataset["Catagory"]==my_cats[1]]

sampling eval data

In [67]:
n_sample = 10

In [68]:
# Sample the eval data from dataset
health_sample = health_set.sample(n=n_sample, random_state=42)
food_sample = food_set.sample(n=n_sample, random_state=42)

# Remove sampled data
health_set = health_set.drop(health_sample.index)
food_set = food_set.drop(food_sample.index)

# Reset index
health_sample = health_sample.reset_index().drop("index", axis=1)
food_sample = food_sample.reset_index().drop("index", axis=1)

make choices

In [69]:
import random

# Make choice and prepare data in format for openthaigpt_eval
# Make choice by sampling from dataset
def make_choices(dataset, sample_set, n_choice=4):
    n_sample = len(sample_set)

    # Copyn the datast so the original one doesn't change
    _dataset = dataset.copy()
    _sample_set = sample_set.copy()

    # Prepare columns for openthaigpt_eval
    _sample_set["input"], _sample_set["result"] = [None, None]
    _sample_set["isAnswerable"], _sample_set["isMultipleChoice"], _sample_set["isSingleChoiceSolution"], _sample_set["note"] = [True, True, True, None]
    _sample_set["year"] = 2024
    _sample_set["no"] = list(range(1, n_sample+1))

    for i in range(n_sample):
        # Randomly correct_id for the correct answer
        correct_id = random.randint(1, n_choice)
        correct_choice = f"({correct_id}) {_sample_set.loc[i, 'Answer']}"

        # Choose answers which all have different question,answer from the question,answer by re-sampling it
        question = _sample_set.loc[i, "Question"]
        random_row = _dataset.sample(n_choice)
        while (question in random_row["Question"].values) or (correct_choice in random_row["Answer"].values):
            random_row = _dataset.sample(n_choice)

        # Make text choices
        choices = list(random_row["Answer"].values)
        choices = [f"({idx+1}) {choice}" if idx+1 != correct_id else correct_choice for idx, choice in enumerate(choices)]
        choices_text = ' '.join(choices)

        # Update sample_set
        _sample_set.loc[i, "input"] = choices_text
        _sample_set.loc[i, "result"] = f"({correct_id}) คือคำตอบที่ถูกต้อง"

        # Drop the sampled values
        # Update _dataset if length is less than n_choice
        _dataset = _dataset.drop(random_row.index)
        if len(_dataset) < n_choice: _dataset = dataset

    # Rename in format of openthaigpt_eval
    _sample_set = _sample_set.rename(columns={"Question": "instruction"})

    return _sample_set[["year", "no", "instruction", "input", "result", "isAnswerable", "isMultipleChoice", "isSingleChoiceSolution", "note"]]

In [70]:
n_choice = 4
health_eval = make_choices(health_set, health_sample, n_choice)
food_eval = make_choices(food_set, food_sample, n_choice)

save data

In [71]:
import os

save_dir = 'katunyou_exams'
if not os.path.exists(save_dir): os.makedirs(save_dir)

health_eval.to_csv(os.path.join(save_dir, "katunyou_health_exam.csv"), index=False)
food_eval.to_csv(os.path.join(save_dir, "katunyou_food_exam.csv"), index=False)